In [2]:
import time
import cloudscraper
import requests
import os
from decimal import Decimal

WEI_PER_ETHER = Decimal("1000000000000000000")  # 1 Ether = 10^18 Wei


def fetch_all_pets():
    url = "https://msu.io/marketplace/api/marketplace/explore/items"
    fetch_amount = 10
    payload = {
        "filter": {
            "categoryNo": 1000400000,
            "price": {"min": 0, "max": 500},
            "level": {"min": 0, "max": 250},
            "starforce": {"min": 0, "max": 25},
            "potential": {"min": 0, "max": 4},
            "bonusPotential": {"min": 0, "max": 4},
        },
        "sorting": "ExploreSorting_RECENTLY_LISTED",
        "paginationParam": {"pageNo": 1, "pageSize": fetch_amount},
    }

    scraper = cloudscraper.create_scraper()
    response = scraper.post(url, json=payload)
    response.raise_for_status()

    return response.json()["items"]


def get_singal_pet_skill_info(tokenId: int):
    url = f"https://msu.io/marketplace/api/marketplace/items/{tokenId}"

    scraper = cloudscraper.create_scraper()
    response = scraper.get(url)
    response.raise_for_status()
    pet_skills = response.json()["item"]["pet"]["petSkills"]
    return pet_skills

def line_notify(message: str):
    url = 'https://notify-api.line.me/api/notify'
    token = os.getenv('line_notify_token')
    headers = {
        'Authorization': 'Bearer ' + token
    }
    data = {
        'message':message
    }
    data = requests.post(url, headers=headers, data=data)


In [3]:
# 已通知列表ID
notified_list = []

while True:
    all_pets_list = fetch_all_pets()
    for i in all_pets_list:
        tokenId = i["tokenId"]
        pet_skills = set(get_singal_pet_skill_info(tokenId))
        print(pet_skills)
        # 篩選條件
        filters = [{"Expanded Auto Move", "Auto Move"}, {"Auto Buff"}, {"Magnet Effect"}]

        # 檢查每組篩選條件是否符合
        for filter_set in filters:
            # 檢查 filter_set 是否在 pet_skills 中
            if filter_set.issubset(pet_skills):
                price = int(int(i['salesInfo']['priceWei']) / WEI_PER_ETHER)
                formatted_skills = "\n".join(filter_set)
                message = f"\n發現高價值寵物\n價格: ${price}\n技能:\n{formatted_skills}\nhttps://msu.io/marketplace/nft/{tokenId}"
                print(message)
                if tokenId not in notified_list:
                    line_notify(message)
                    notified_list.append(tokenId)

        time.sleep(0.2)


    time.sleep(15)

{'NESO Magnet', 'Item Pouch', 'Auto MP Potion Pouch', 'Auto HP Potion Pouch'}
{'Item Pouch', 'Auto MP Potion Pouch', 'Expanded Auto Move', 'Auto Move', 'NESO Magnet', 'Auto HP Potion Pouch'}

發現高價值寵物
價格: $85
技能:
Expanded Auto Move
Auto Move
https://msu.io/marketplace/nft/8191675398995188881079838358018
{'Item Pouch', 'Auto MP Potion Pouch', 'Expanded Auto Move', 'NESO Magnet', 'Auto HP Potion Pouch'}
{'NESO Magnet', 'Item Pouch', 'Auto MP Potion Pouch', 'Auto HP Potion Pouch'}
{'Item Pouch', 'Auto MP Potion Pouch', 'Expanded Auto Move', 'NESO Magnet', 'Auto HP Potion Pouch'}
{'NESO Magnet', 'Item Pouch', 'Auto MP Potion Pouch', 'Auto HP Potion Pouch'}
{'NESO Magnet', 'Item Pouch', 'Auto MP Potion Pouch', 'Auto HP Potion Pouch'}
{'Item Pouch', 'Auto MP Potion Pouch', 'Expanded Auto Move', 'NESO Magnet', 'Fatten Up', 'Auto HP Potion Pouch'}
{'Item Pouch', 'Auto MP Potion Pouch', 'Auto Move', 'NESO Magnet', 'Auto HP Potion Pouch'}
{'Item Pouch', 'Auto MP Potion Pouch', 'Expanded Auto Move

KeyboardInterrupt: 